### Yelp Dataset 10Mar2025 Validations or Proof of Concepts
#### 1. Extrat Tar File

In [1]:
# !pip3 install -r requirements.txt

In [2]:
from src import modules as f
%load_ext autoreload
%autoreload 2

In [3]:
# load for the first time to set up files
# f.json_2_parquet()

In [4]:
import os
from pyspark.sql import SparkSession

# Set the correct network IP if necessary
os.environ["SPARK_LOCAL_IP"] = "192.168.5.29"  # or "192.168.5.29" if explicitly needed

# Build Spark session WITHOUT RAPIDS
spark = SparkSession.builder \
    .appName("SparkNoGPU") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to reduce verbosity
spark.sparkContext.setLogLevel("ERROR")

print("Spark session created!")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 11:49:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark session created!


#### 2. Import JSON Tables

In [ ]:
# Load each Parquet file into a Spark DataFrame
business_df = spark.read.parquet(os.path.join(parquet_dir, 'business.parquet')).sample(False, 0.1, seed=42)
# review_df   = spark.read.parquet(os.path.join(parquet_dir, 'review.parquet')).sample(False, 0.1, seed=42)
# checkin_df  = spark.read.parquet(os.path.join(parquet_dir, 'checkin.parquet')).sample(False, 0.1, seed=42)
# tip_df      = spark.read.parquet(os.path.join(parquet_dir, 'tip.parquet')).sample(False, 0.1, seed=42)
# user_df     = spark.read.parquet(os.path.join(parquet_dir, 'user.parquet')).sample(False, 0.1, seed=42)

# Show a sample from one DataFrame
business_df.show(5)

In [ ]:
spark.stop()